In [48]:
!pip install google-generativeai PyMuPDF sentence-transformers scikit-learn

In [49]:
import os
import fitz
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from typing import List
import numpy as np
import textwrap
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
os.environ["GOOGLE_API_KEY"] = "GOOGLE_API_KEY"
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [51]:
from google.colab import files
uploaded = files.upload()

filename = list(uploaded.keys())[0]

def extract_text(path):
    text = ""
    with fitz.open(path) as doc:
        for page in doc:
            text += page.get_text()
    return text

document_text = extract_text(filename)
print("Document loaded successfully.")

Saving Acko-Group-Health-Insurance.pdf to Acko-Group-Health-Insurance (4).pdf
Document loaded successfully.


In [72]:
import nltk
nltk.download("punkt_tab")

from nltk.tokenize import sent_tokenize

def split_text(text, max_tokens=300, chunk_size=300, chunk_overlap=50):
    sentences = sent_tokenize(text)
    chunks = []
    curr_chunk = []
    curr_length = 0

    for sentence in sentences:
        token_count = len(sentence.split())
        if curr_length + token_count > max_tokens:
            chunks.append(" ".join(curr_chunk))
            curr_chunk = []
            curr_length = 0
        curr_chunk.append(sentence)
        curr_length += token_count

    if curr_chunk:
        chunks.append(" ".join(curr_chunk))

    return chunks

chunks = split_text(document_text)
print(f"Total chunks created: {len(chunks)}")

Total chunks created: 173


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [73]:
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(chunks, show_progress_bar=True)
print("Embedding complete.")

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Embedding complete.


In [ ]:
import google.generativeai as genai
import textwrap
from sklearn.metrics.pairwise import cosine_similarity

genai.configure(api_key="GOOGLE_API_KEY")
gen_model = genai.GenerativeModel(model_name="gemini-2.0-flash")

def retrieve_relevant_chunks(query, top_k=3):
    query_embedding = model.encode([query])
    similarities = cosine_similarity(query_embedding, embeddings)[0]
    top_indices = similarities.argsort()[-top_k:][::-1]
    return [chunks[i] for i in top_indices]

def ask_gemini(question):
    relevant_chunks = retrieve_relevant_chunks(question)
    context = "\n\n".join(relevant_chunks)

    prompt = f"""Use the following document information to answer the question.
    Context: {context}
    Question: {question}
    Answer:"""

    response = gen_model.generate_content(prompt, generation_config=genai.types.GenerationConfig(temperature=0.2))
    return str(response.text).strip()

sample_questions = ["Can you explain the Cancellation of the Policy?"]

for question in sample_questions:
    print("Question:", question)
    answer = ask_gemini(question)
    print("Answer:\n", textwrap.fill(answer))

Question: Can you explain the Cancellation of the Policy?
Answer:
 The document outlines two ways the policy can be cancelled:  **1.
Cancellation by You (the Policyholder) - Section 4.23:**  *   You can
cancel the policy by giving Acko a **15-day written notice.** *   If
no claims have been made under the policy, Acko will refund a
percentage of the premium for the unexpired policy period, after
deducting their expenses. The refund amount is based on a "short
period scale" (not detailed here, but implied to exist). *
**Important:** The "short period scale" applies only to single premium
policies. *   For instalment premium policies, Acko will refund the
paid premium on a pro rata basis, after deducting their expenses.
**2. Termination by Acko (the Insurer) - Section 4.24:**  Acko can
terminate the policy under the following circumstances, giving **15
days' notice in writing by Registered Post Acknowledgment Due /
recorded delivery to Your last known address:**  *
**Misrepresentation, f